In [3]:
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
import random
from collections import Counter

from generator import generateCliffordCircuit

In [4]:
def stringToQiskitSingleGate(gateString, qiskitCir, whichQubit):
    if gateString == 'I':
        qiskitCir.id(whichQubit)
    elif gateString == 'X':
        qiskitCir.x(whichQubit)
    elif gateString == 'Y':
        qiskitCir.y(whichQubit)
    elif gateString == 'Z':
        qiskitCir.z(whichQubit)
    elif gateString == 'H':
        qiskitCir.h(whichQubit)
    elif gateString == 'S':
        qiskitCir.s(whichQubit)
        
def transpileListToQiskitCircuit(cir):
    depth = len(cir)
    width = len(cir[0])
    qiskitCir = qk.QuantumCircuit(width)
    for d in range(width):
        if d % 2 == 0:
            for w in range(width):
                singleGate = cir[d][w]
                stringToQiskitSingleGate(singleGate, qiskitCir, w)
            if d != width - 1:
                qiskitCir.barrier()
        else:
            c = cir[d].index('CNOT_C')
            t = cir[d].index('CNOT_T')
            qiskitCir.cx(c, t)
            if d != width - 1:
                qiskitCir.barrier()
    return qiskitCir

In [5]:
width = 5 # num_qubits
depth = 5 # number of layers in the circuit (easy + hard)
singleGateSet = ['X', 'H', 'Z', 'I', 'S']
doubleGateSet = ['CNOT_C', 'CNOT_T']
twirlingGateSet = ['X','Y' 'Z', 'I']

circuit = generateCliffordCircuit(width, depth, singleGateSet, doubleGateSet)
qiskitCircuit = transpileListToQiskitCircuit(circuit)
print(circuit)
if depth < 10:
    print(qiskitCircuit)

[['Z', 'I', 'I', 'Z', 'Z'], ['CNOT_T', 'I', 'I', 'I', 'CNOT_C'], ['S', 'X', 'S', 'H', 'H'], ['I', 'CNOT_C', 'CNOT_T', 'I', 'I'], ['H', 'Z', 'S', 'I', 'Z']]
     ┌───┐ ░ ┌───┐ ░ ┌───┐ ░       ░ ┌───┐
q_0: ┤ Z ├─░─┤ X ├─░─┤ S ├─░───────░─┤ H ├
     ├───┤ ░ └─┬─┘ ░ ├───┤ ░       ░ ├───┤
q_1: ┤ I ├─░───┼───░─┤ X ├─░───■───░─┤ Z ├
     ├───┤ ░   │   ░ ├───┤ ░ ┌─┴─┐ ░ ├───┤
q_2: ┤ I ├─░───┼───░─┤ S ├─░─┤ X ├─░─┤ S ├
     ├───┤ ░   │   ░ ├───┤ ░ └───┘ ░ ├───┤
q_3: ┤ Z ├─░───┼───░─┤ H ├─░───────░─┤ I ├
     ├───┤ ░   │   ░ ├───┤ ░       ░ ├───┤
q_4: ┤ Z ├─░───■───░─┤ H ├─░───────░─┤ Z ├
     └───┘ ░       ░ └───┘ ░       ░ └───┘


In [6]:
def Clifford_Permute(cliff,pauli):
    '''This function permutes the pauli operators according to the clifford group, so it computes 
    P' = C P C^T, where C is a clifford operator and P is a pauli operator.
    Inputs: cliff - A Qiskit Circuit, Clifford, or Gate object.
            pauli - A Qiskit Pauli object.
    Outputs: new_pauli - A Qiskit Pauli object.'''

    return pauli.evolve(cliff,frame="s")

In [7]:
Clifford_Permute(qiskitCircuit, qk.quantum_info.Pauli('X'*width))

Pauli('ZZZYI')

In [12]:
def build_A_2(num_rows=10, num_columns=10, curr_pauli ='X'):
    '''This function builds the A matrix for the linear system of equations, Ax=b, where x is the vector of 
    pauli expectation values. The A matrix is built by applying the clifford group to the pauli operators.
    Inputs: num_rows - The number of rows in the A matrix.
            num_columns - The number of columns in the A matrix.
            curr_clifford - The clifford to apply to the pauli operators.
    Outputs: A - A numpy array representing the A matrix.'''
    A = np.zeros((num_rows,num_columns),dtype=np.complex128)
    for i in range(num_rows):
        for j in range(num_columns):
            pauli = qk.quantum_info.Pauli('I'*width)
            for k in range(width):
                if j & (1 << k):
                    pauli = pauli.tensor(qk.quantum_info.Pauli('Z'))
                else:
                    pauli = pauli.tensor(qk.quantum_info.Pauli('X'))
            A[i,j] = Clifford_Permute(curr_pauli,pauli).to_matrix()[0,0]
        curr_pauli = qk.quantum_info.random_clifford(width).to_matrix()
    return A

In [13]:
build_A_2()

QiskitError: 'Incorrect number of qubits for Clifford circuit (1 != 10).'